### Table of Contents

* [1: Importing, Merging, and Cleaning the Data](#import_merge_clean_1)
    * [1.1: Import](#import_1_1)
    * [1.2: Merge](#merge_1_2)
        * [1.2.1: Merge HCPCS Codes](#merge_hcpcs_1_2_1)
        * [1.2.2: Merge Carrier Codes](#merge_carrier_1_2_2)
        * [1.2.3: Merge Locality Codes](#merge_locality_1_2_3)
    * [1.3: Clean](#clean_1_3)
* [2: Visualization](#vis_2)
    * [2.1: Nation-Wide](#vis_nation_2_1)
    * [2.2: Statewide](#vis_state_2_2)
* [3: Business Application](#business_app_3)
    * [3.1: Revenue Forecasting](#business_app_rev_3_1)
    * [3.2: Statewide](#business_app_rev_3_2)

### 1 - Importing, Merging, and Cleaning the Data<a class="anchor" id="import_merge_clean_1"></a>

#### 1.1 - Import<a class="anchor" id="import_1_1"></a>
* Download Ambulance Fee Schedule from [The Centers for Medicare & Medicaid Services](https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/), CMS

In [1]:
# List of urls to Ambulance Fee Schedule data
lst_url = ['https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/afs_2005.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2006_afs.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2007_afs.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2008_Ambulance_Fee_Schedule_PUF_update.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2009_AFS_PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2010_AFS_PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2011_AFS_PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2012_AFS_PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2013-AFS-PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2014-AFS-PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2015-AFS-PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2016-AFS-PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/CY-2017-File.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2018-AFS-PUF.zip',
          'https://www.cms.gov/Medicare/Medicare-Fee-for-Service-Payment/AmbulanceFeeSchedule/Downloads/2019-AFS-PUF.zip',
          'https://www.cms.gov/files/zip/cy-2020-file.zip']

# Import the data using the wget package
# !pip install wget
import wget
for url in lst_url:
    wget.download(url)
    file_name = url.split('/')[-1]
    print(f'Downloaded: {file_name}')
    
# Get list of zip files in current directory (name changes from website to here)
import os
zip_files = [f for f in os.listdir('.') if os.path.isfile(f) and f[-3:]=='zip'];

# Extract zip files to data folder
import zipfile
for file in zip_files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall('data')
        
# Delete zip files
for file in zip_files:
    os.remove(file)

# Rename relevant spreadsheets
os.rename('./data/' + 'ambfspuf_2013_2ndrevisedfinal.xlsx', './data/' + '2013.xlsx')
os.rename('./data/' + 'ambfspuf 1Q2014_ext_finalv2.xlsx', './data/' + '2014.xlsx')
os.rename('./data/' + 'ambfspuf_2016_.xlsx', './data/' + '2016.xlsx')

# Delete non txt files
non_txt_files = [f for f in os.listdir('./data/') if f[-3:]!='txt']
txt_files = [f for f in os.listdir('./data/') if f[-3:]=='txt']
keep_files = ['2013.xlsx', '2014.xlsx', '2016.xlsx']
for file in non_txt_files:
    if file not in keep_files:
        os.remove('./data/' + file)

# Rename txt files for redability
txt_files_rename = []
for file in txt_files:
    s = ''.join([i for i in file if i.isnumeric()])
    idx_yr = s.find('2')
    s = s[idx_yr:idx_yr + 4] + '.txt'
    txt_files_rename.append(s)
txt_files_rename
for i in range(len(txt_files)):
    os.rename('./data/' + txt_files[i], './data/' + txt_files_rename[i])
os.remove('./data/8.txt')
os.remove('./data/2013.txt')
os.remove('./data/2014.txt')
os.remove('./data/2016.txt')

Downloaded: afs_2005.zip
Downloaded: 2006_afs.zip
Downloaded: 2007_afs.zip
Downloaded: 2008_Ambulance_Fee_Schedule_PUF_update.zip
Downloaded: 2009_AFS_PUF.zip
Downloaded: 2010_AFS_PUF.zip
Downloaded: 2011_AFS_PUF.zip
Downloaded: 2012_AFS_PUF.zip
Downloaded: 2013-AFS-PUF.zip
Downloaded: 2014-AFS-PUF.zip
Downloaded: 2015-AFS-PUF.zip
Downloaded: 2016-AFS-PUF.zip
Downloaded: CY-2017-File.zip
Downloaded: 2018-AFS-PUF.zip
Downloaded: 2019-AFS-PUF.zip
Downloaded: cy-2020-file.zip


#### Create dataframe for each year

In [2]:
# Clean data and create dataframe for each year
import pandas as pd

# Create list of years with associated txt files
lst_txt = [2009, 2010, 2011, 2015, 2012, 2017, 2018, 2019, 2020]

# Create list of years with associated xlsx files
lst_xlsx = [2013, 2014, 2016,]

# Create pandas df for each txt file / year
for yr in lst_txt:
    df_name = 'df_' + str(yr)
    globals()[df_name] = pd.read_csv('./data/' + str(yr) + '.txt', sep="\t", dtype={'CONTRACTOR/CARRIER': str,'contractor/carrier': str})
    globals()[df_name] = globals()[df_name].iloc[:, : 10]
    
# Create pandas df for each xlsx file / year
for yr in lst_xlsx:
    df_name = 'df_' + str(yr)
    globals()[df_name] = pd.read_excel('./data/' + str(yr) + '.xlsx')
    globals()[df_name] = globals()[df_name].iloc[:, : 10]

# Create list of all dataframes
lst_df = [df_2009,df_2010,df_2011,
          df_2012,df_2013,df_2014,
          df_2015,df_2016,df_2017,
          df_2018,df_2019,df_2020]

# Create list of column names to superimpose on each df for consistency
col_consistent = ['contractor/carrier', 'locality', 'hcpcs', 'rvu', 'gpci', 'base rate', 'urban base rate / urban mileage', 'rural base rate / rural mileage', 'rural base rate / lowest quartile', 'rural ground miles']

# Assign column names to each dataframe
for i in range(len(lst_df)):
    lst_df[i].columns = col_consistent
    # Convert locality to float
    lst_df[i]['locality'] = lst_df[i]['locality'].astype(float)
    # Found ID errors in data
    lst_df[i]['contractor/carrier'] = lst_df[i]['contractor/carrier'].astype(str).str.replace('01112', '01102').str.replace('10112', '10102').str.replace('10212', '10202').str.replace('10312', '10302')
    # Remove $ signs and convert to float
    lst_df[i]['base rate'] = lst_df[i]['base rate'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
    lst_df[i]['urban base rate / urban mileage'] = lst_df[i]['urban base rate / urban mileage'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
    lst_df[i]['rural base rate / rural mileage'] = lst_df[i]['rural base rate / rural mileage'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
    lst_df[i]['rural base rate / lowest quartile'] = lst_df[i]['rural base rate / lowest quartile'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
    lst_df[i]['rural ground miles'] = lst_df[i]['rural ground miles'].astype(str).str.replace('$', '').str.replace(',', '').astype(float)
    
# Some dataframes have labels for various states and NA values following. Delete these rows
for i in range(len(lst_df)):
    lst_df[i] = lst_df[i].loc[~lst_df[i]['locality'].isna()]

#### 1.2 - Merge<a class="anchor" id="merge_1_2"></a>

##### 1.2.1 - Merge HCPCS Codes<a class="anchor" id="merge_hcpcs_1_2_1"></a>
* "HCPCS is a collection of standardized codes that represent medical procedures, supplies, products and services." [source](https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/HCPCS/index.html#:~:text=HCPCS%20is%20a%20collection%20of,by%20Medicare%20and%20other%20insurers.&text=HCPCS%20is%20divided%20into%20two%20subsystems%2C%20Level%20I%20and%20Level%20II.)
* Join Alpha-Numeric HCPCS Codes (i.e. all the different activities to be billed) [download](https://www.cms.gov/Medicare/Coding/HCPCSReleaseCodeSets/Alpha-Numeric-HCPCS)

In [3]:
# Import HCPCS Code dataframe
df_code = pd.read_excel('./_hcpcs/HCPCS_2020.xlsx')

# Lowercase column names
df_code.columns = df_code.columns.str.lower()

# Merge HCPCS dataframe with main dataframe
lst_yrs = [yr for yr in range(2009,2020+1)]
for i in range(len(lst_yrs)):
    lst_df[i] = pd.merge(lst_df[i], df_code[['hcpc/mod', 'short description', 'long description']], how='left', left_on='hcpcs', right_on='hcpc/mod')

##### 1.2.2 - Merge Carrier Codes<a class="anchor" id="merge_carrier_1_2_2"></a>
* Import contractor/carrier code data and perform string manipulation

In [4]:
# Loop through 2009-2016 years, save as dataframe, and performe string manipulation
for yr in lst_yrs:
    df_name = 'df_carrier_code_' + str(yr)
    try:
        globals()[df_name] = pd.read_csv('./_index/' + str(yr) + '.csv', header=None)
    except:
        globals()[df_name] = pd.read_excel('./_index/' + str(yr) + '.xlsx', header=None)
    
    # new data frame with split value columns 
    split_columns = globals()[df_name][0].str.split(" ", n = 1, expand = True)
    
    # making separate contractor/carrier column from new data frame 
    globals()[df_name]["contractor/carrier"] = split_columns[0]
    globals()[df_name]["contractor/carrier"] = globals()[df_name]["contractor/carrier"].astype(str).str.replace('U', '').str.strip()
    # Found ID errors in data
    globals()[df_name]['contractor/carrier'] = globals()[df_name]['contractor/carrier'].str.replace('0632', '06302').str.replace('01112', '01102')
    
    # making separate location column from new data frame 
    globals()[df_name]["contractor/carrier name"]= split_columns[1].str.replace('\d+', '').str.replace('-', '').str.strip().str.replace('-', '').str.lower()
    
    # reducing number of columns in data frame
    globals()[df_name] = globals()[df_name][["contractor/carrier", "contractor/carrier name"]]

# Create list of dataframes for convenient merging
lst_carrier_df = [df_carrier_code_2009,df_carrier_code_2010,df_carrier_code_2011,
                  df_carrier_code_2012,df_carrier_code_2013,df_carrier_code_2014,
                  df_carrier_code_2015,df_carrier_code_2016,df_carrier_code_2017,
                  df_carrier_code_2018,df_carrier_code_2019,df_carrier_code_2020]

#  Merge the datasets
for i in range(len(lst_yrs)):
    lst_df[i] = pd.merge(lst_df[i], lst_carrier_df[i], how='left', left_on='contractor/carrier', right_on='contractor/carrier')

##### 1.2.3 - Merge Locality Codes<a class="anchor" id="merge_locality_1_2_3"></a>

In [6]:
import numpy as np

# Create pandas df for each txt file / year
for yr in lst_yrs:
    df_name = 'df_' + str(yr) + '_locality'
    globals()[df_name] = pd.read_csv('./_locality/RVU' + str(yr)[-2:] + '/' + str(yr)[-2:] + 'LOCCO.csv')
    # Drop rows with all NA or all ' '
    globals()[df_name].replace(' ', np.NaN, inplace=True)
    globals()[df_name].dropna(how = 'all', inplace = True)
    # Convert locality number to float for merging
    globals()[df_name]['Locality Number'] = globals()[df_name]['Locality Number'].astype(float)
    # Need Carrier Codes to be 5 in length
    globals()[df_name]['Carrier Number'] = '000000' + globals()[df_name]['Carrier Number']
    globals()[df_name]['Carrier Number'] = globals()[df_name]['Carrier Number'].str[-5:]
    # Found ID errors in data
    globals()[df_name]['Carrier Number'] = globals()[df_name]['Carrier Number'].astype(str).str.replace('01112', '01102').str.replace('10112', '10102').str.replace('10212', '10202').str.replace('10312', '10302').str.replace('00590', '09102').str.replace('00973', '09202')
    # Convert column names to lower case
    globals()[df_name].columns = [i.lower() for i in globals()[df_name].columns]
    # Rename columns
    globals()[df_name].rename(columns={"fee schedule area ": "locality name"}, inplace=True)
    # Convert Locality Name to lowercase
    globals()[df_name]['locality name'] = globals()[df_name]['locality name'].str.lower()

lst_locality_df = [df_2009_locality,df_2010_locality,df_2011_locality,
                   df_2012_locality,df_2013_locality,df_2014_locality,
                   df_2015_locality,df_2016_locality,df_2017_locality,
                   df_2018_locality,df_2019_locality,df_2020_locality]

#  Merge the datasets
for i in range(len(lst_yrs)):
    lst_df[i] = pd.merge(lst_df[i], lst_locality_df[i], how='left', left_on=['contractor/carrier', 'locality'], right_on=['carrier number', 'locality number'])

#### 1.3 - Clean<a class="anchor" id="clean_1_3"></a>

In [7]:
# Create list of column names for an intuitive reordering
col_reorder = ['year','contractor/carrier','contractor/carrier name',
               'locality','locality name', 'hcpcs','short description','long description',
               'rvu','gpci','base rate','urban base rate / urban mileage',
               'rural base rate / rural mileage','rural base rate / lowest quartile',
               'rural ground miles']

# Reorder columns
for i in range(len(lst_yrs)):
    lst_df[i]['year'] = lst_yrs[i]
    lst_df[i] = lst_df[i][col_reorder]
    
# Compile all dataframes
df = pd.concat(lst_df)
df.sample(3)

,year,contractor/carrier,contractor/carrier name,locality,locality name,hcpcs,short description,long description,rvu,gpci,base rate,urban base rate / urban mileage,rural base rate / rural mileage,rural base rate / lowest quartile,rural ground miles
98,2010,00835,oregon,99.0,rest of oregon,A0427,Als1-emergency,"Ambulance service, advanced life support, emer...",209.65,1.90,0.964,396.060000,399.950000,490.330,NaN
805,2016,11502,north carolina,0.0,statewide,A0426,Als 1,"Ambulance service, advanced life support, non-...",220.74,1.20,0.930,256.946658,259.465743,318.105,NaN
908,2009,12402,new jersey,1.0,northern nj,A0433,Als 2,"Advanced life support, level 2 (als 2)",212.13,2.75,1.228,689.990000,696.760000,854.230,NaN


### 2 - Visualization<a class="anchor" id="vis_2"></a>

#### In order to proceed with our analysis let us define the features in our data
* *Year* - Year of fee schedule

* *Contractor/Carrier* - Entity which has the responsibility for adjudicating and paying claims within a defined geographical location

* *Locality* - This field represents subsets of locations within a defined jurisdiction with different GPCI

* *HCPCS* - This field has the full range of HCPCS services payable under the ambulance fee schedule
    * "HCPCS is a collection of standardized codes that represent medical procedures, supplies, products and services." [source](https://www.nlm.nih.gov/research/umls/sourcereleasedocs/current/HCPCS/index.html#:~:text=HCPCS%20is%20a%20collection%20of,by%20Medicare%20and%20other%20insurers.&text=HCPCS%20is%20divided%20into%20two%20subsystems%2C%20Level%20I%20and%20Level%20II.)
    
* *Relative Value Unit (RVU)* - Ratio from 1 - 3.25 rating the complexity or cost of a given ambulance ride (i.e. a normal ambulance ride has an RVU of 1). The RVU Values are:
    * BLS (A0428) 1.00 
    * ALS1 (A0426) 1.20 
    * BLS - Emergency (A0429) 1.60 
    * PI (A0432) 1.75
    * ALS1- Emergency (A0427) 1.90 
    * ALS2 (A0433) 2.75 
    * SCT (A0434) 3.25 
    
* *Geographic Practice Cost Index (GPCI)* - "Inflation factor" to adjust payment to account for regional differences

* *Base Rate* - The Base Rate is a nationally uniform “base” amount used to calculate each HCPCS’ payment amount

* *Urban Base Rate / Mileage* - Equation that calculates urban fee rate based on RVU, GPCI, and National Base Rate

* *Rural Base Rate / Mileage* - Equation that calculates rural fee rate based on RVU, GPCI, and National Base Rate

* *Rural Base Rate / Lowest Quartile* - This field displays the rural base rate adjusted by the “super-rural bonus” multiplier of 22.6%. This is to incentivize medical services in sparsely populated areas

* *Rural Ground Miles* - This field displays the amounts for rural ground miles 1-17


#### 2.1 - Nation-Wide<a class="anchor" id="vis_nation_2_1"></a>

In [8]:
df.loc[df['contractor/carrier name'] == 'texas']['locality name'].unique()

array(['brazoria', 'dallas', 'galveston', 'houston', 'beaumont',
       'fort worth', 'austin', 'rest of state ', 'brazoria, tx',
       'dallas, tx', 'galveston, tx', 'houston, tx', 'beaumont, tx',
       'fort worth, tx', 'austin, tx', 'rest of texas'], dtype=object)

#### 2.2 - Statewide<a class="anchor" id="vis_state_2_2"></a>

### 3 - Business Application<a class="anchor" id="business_app_3"></a>

#### 3.1 - Revenue Forecasting<a class="anchor" id="business_app_rev_3_1"></a>

#### 3.2 - Statewide<a class="anchor" id="business_app_rev_3_2"></a>